# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [389]:
from random import random, seed
from itertools import product
import numpy as np
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [390]:
UNIVERSE_SIZE = [100, 1000, 10000, 100000, 100000, 100000] 
NUM_SETS = [10, 100, 1000, 10000, 10000, 10000] 
DENSITY = [0.2, 0.2, 0.2, 0.1, 0.2, 0.3]

# rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

## Helper Functions

In [391]:
# Checks whether the solution is valid (i.e., covers all elements of the universe)
def valid(SETS, solution):
    # Use logical OR across the rows of SETS that are marked as True in the solution
    return np.all(np.logical_or.reduce(SETS[solution]))

# Returns the cost of a solution, which is to be minimized
def cost(COSTS, solution):
    # Sum the costs corresponding to the selected sets in the solution
    return COSTS[solution].sum()

# Returns the fitness of a solution as a tuple
def fitness(instance, solution):
    (SETS, COSTS) = instance
    # Fitness includes validity (True if all elements are covered) and the negative cost
    return (valid(SETS, solution), -cost(COSTS, solution))

# Generates sets and costs based on the specified parameters
def generate_sets_and_costs(UNIVERSE_SIZE, NUM_SETS, DENSITY):
    # Create a random matrix of booleans indicating which sets cover which elements of the universe
    SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
    
    # Ensure every element in the universe is covered by at least one set
    for s in range(UNIVERSE_SIZE):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(NUM_SETS), s] = True
            
    # Calculate costs for each set based on the number of elements it covers
    COSTS = np.pow(SETS.sum(axis=1), 1.1)  # Increase cost with more coverage
    return (SETS, COSTS)

# Generates a random starting solution that is guaranteed to be valid
def generate_starting_solution(i, instance):
    # Create a random solution where each set is included with a probability of 0.5
    solution = np.random.random(NUM_SETS[i]) < 0.5
    # Regenerate the solution until it is valid (covers all elements of the universe)
    while not valid(instance[0], solution):
        solution = np.random.random(NUM_SETS[i]) < 0.5
    return solution

# Plots the fitness history over iterations
def plot_fitness(history, title): 
    plt.figure(figsize=(14, 8))  # Set figure size
    plt.plot(
       range(len(history)),
       list(accumulate(history, max)),  # Plot cumulative max fitness values
       color="red",
       label="Negative Cost (Cumulative)"
    )
    # Scatter plot of individual fitness values
    _ = plt.scatter(range(len(history)), history, marker=".", label="Negative Cost")
    plt.xlabel("Iteration")
    plt.ylabel("Fitness Value")
    plt.title(title)
    plt.legend()
    plt.show()

---

## Solution

### Random-Mutation Hill Climber with Single Mutation

In [392]:
# Generates a valid neighbor solution by tweaking the current one
def single_mutation(i, solution):
    new_solution = solution.copy()  # Create a copy of the current solution
    index = np.random.randint(NUM_SETS[i])  # Randomly select an index to mutate
    new_solution[index] = ~new_solution[index]  # Flip the boolean value at the selected index
    return new_solution


# Performs hill climbing optimization to find a better solution
def hill_climber(i, instance, tweak, starting_solution, MAX_STEPS):
    solution = starting_solution.copy()  # Initialize the current solution
    history = [float(fitness(instance, solution)[1])]  # Track fitness over iterations

    for _ in range(MAX_STEPS):
        new_solution = tweak(i, solution)  # Generate a new solution
        history.append(float(fitness(instance, new_solution)[1]))  # Record fitness of the new solution
        
        current_fitness = fitness(instance, solution)  # Get fitness of the current solution
        new_fitness = fitness(instance, new_solution)  # Get fitness of the new solution

        # If the new solution has higher fitness, update the current solution
        if new_fitness > current_fitness:
            solution = new_solution
            current_fitness = new_fitness
    
    # Plot fitness history for the hill climbing process
    plot_fitness(history, f"Hill Climbing for {UNIVERSE_SIZE[i]} universe size, {NUM_SETS[i]} sets and density {DENSITY[i]}")

    return solution

In [ ]:
# Parameter for Hill Climbing
MAX_STEPS = 1000

# Run Hill Climbing with single mutation for different instances
for i in range(6):
    # Generate sets and costs for the current universe size and density
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    # Generate a valid starting solution for the current instance
    starting_solution = generate_starting_solution(i, instance)
    # Perform hill climbing optimization using single mutation
    solution = hill_climber(i, instance, single_mutation, starting_solution, MAX_STEPS)
    # Print or log the results: universe size, number of sets, density, validity, and cost of the solution
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))

### Random-Mutation Hill Climber with Multiple Mutations

In [394]:
# Generates a valid neighbor solution by tweaking the current one multiple times
def multiple_mutations(i, solution):
    new_solution = solution.copy()  # Create a copy of the current solution
    index = None  # Initialize index variable

    # Randomly mutate sets while a certain condition is met
    while index is None or np.random.random() < 0.4:
        index = np.random.randint(NUM_SETS[i])  # Randomly select an index to mutate
        new_solution[index] = ~new_solution[index]  # Flip the boolean value at the selected index
        
    return new_solution  # Return the mutated solution

In [ ]:
# Run Hill Climbing with multiple mutations
for i in range(6):
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    starting_solution = generate_starting_solution(i, instance)
    solution = hill_climber(i, instance, multiple_mutations, starting_solution, MAX_STEPS)
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))

---

### Random-Mutation Hill Climber with Single and Multiple Mutations, starting from a Greedy solution

In [396]:
# Generates a greedy solution for the set cover problem
def greedy_solution(i, instance):
    SETS, _ = instance  # Extract the sets from the instance
    covered = np.full(UNIVERSE_SIZE[i], False)  # Track which elements are covered
    solution = np.full(NUM_SETS[i], False)  # Initialize the solution array

    # Continue until all elements in the universe are covered
    while not valid(SETS, solution):
        best_coverage = -1  # Initialize best coverage found
        best_set = -1  # Initialize the best set index

        # Evaluate each set to find the one that covers the most uncovered elements
        for s in range(NUM_SETS[i]):
            if not solution[s]:  # Only consider sets that are not yet included
                new_covered = np.logical_and(SETS[s], ~covered)  # Determine new coverage
                coverage = np.sum(new_covered)  # Count the number of new elements covered
                
                # Update best coverage and set if the current one is better
                if coverage > best_coverage:
                    best_coverage = coverage
                    best_set = s
        
        # Include the best set in the solution
        solution[best_set] = True
        # Update the covered elements with the newly included set
        covered = np.logical_or(SETS[best_set], covered)

    return solution

In [ ]:
# Run Hill Climbing with single mutations, starting from a greedy solution
for i in range(6):
    # Generate sets and costs for the current universe size and density
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    # Generate a starting solution using the greedy approach
    starting_solution = greedy_solution(i, instance)
    # Perform hill climbing optimization using single mutation
    solution = hill_climber(i, instance, single_mutation, starting_solution, MAX_STEPS)
    # Print or log the results: universe size, number of sets, density, validity, and cost of the solution
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))


In [ ]:
# Run Hill Climbing with multiple mutations, starting from a greedy solution
for i in range(6):
    # Generate sets and costs for the current universe size and density
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    # Generate a starting solution using the greedy approach
    starting_solution = greedy_solution(i, instance)
    # Perform hill climbing optimization using multiple mutations
    solution = hill_climber(i, instance, multiple_mutations, starting_solution, MAX_STEPS)
    # Print or log the results: universe size, number of sets, density, validity, and cost of the solution
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))


---

### Simulated Annealing with Single Mutation

In [399]:
# Performs simulated annealing optimization to find a solution
def simulated_annealing(i, instance, tweak, starting_solution, initial_temp, final_temp, alpha):
    solution = starting_solution  # Initialize the current solution
    temperature = initial_temp  # Set the initial temperature
    history = [float(fitness(instance, solution)[1])]  # Track the fitness over iterations

    # Continue until the temperature is below the final threshold
    while temperature > final_temp:
        new_solution = tweak(i, solution)  # Generate a new solution by tweaking the current one
        history.append(float(fitness(instance, new_solution)[1]))  # Record the fitness of the new solution

        # Check if the new solution is better; accept it if so
        if fitness(instance, new_solution) > fitness(instance, solution):
            solution = new_solution
        elif valid(instance[0], new_solution):  # Check if the new solution is valid
            # Calculate acceptance probability for worse solutions
            acceptance_probability = np.exp((cost(instance[0], solution) - cost(instance[1], new_solution)) / (temperature * NUM_SETS[i]))
            # Accept the new solution based on probability
            if np.random.random() < acceptance_probability:
                solution = new_solution
        
        # Lower the temperature to reduce the probability of accepting worse solutions over time
        temperature *= alpha

    # Plot the fitness history for the simulated annealing process
    plot_fitness(history, f"Simulated Annealing for {UNIVERSE_SIZE[i]} universe size, {NUM_SETS[i]} sets and density {DENSITY[i]}")

    return solution

In [ ]:
# Parameters for Simulated Annealing
initial_temp = 1000  # Initial temperature for the annealing process
final_temp = 0.001   # Final temperature threshold for stopping
alpha = 0.995        # Cooling factor for reducing the temperature

# Run Simulated Annealing with single mutation
for i in range(6):
    # Generate sets and costs for the current universe size and density
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    # Generate a valid starting solution for the current instance
    starting_solution = generate_starting_solution(i, instance)
    # Perform simulated annealing optimization using single mutation
    solution = simulated_annealing(i, instance, single_mutation, starting_solution, initial_temp, final_temp, alpha)
    # Print or log the results: universe size, number of sets, density, validity, and cost of the solution
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))

### Simulated Annealing with Multiple Mutations

In [ ]:
# Run Simulated Annealing with multiple mutations for different instances
for i in range(3,6):
    # Generate sets and costs for the current universe size and density
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    # Generate a valid starting solution for the current instance
    starting_solution = generate_starting_solution(i, instance)
    # Perform simulated annealing optimization using multiple mutations
    solution = simulated_annealing(i, instance, multiple_mutations, starting_solution, initial_temp, final_temp, alpha)
    # Print or log the results: universe size, number of sets, density, validity, and cost of the solution
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))

---

### Simulated Annealing with Single and Multiple Mutations, starting from a Greedy solution

In [ ]:
# Run Simulated Annealing with single mutation, starting from a greedy solution
for i in range(6):
    # Generate sets and costs for the current universe size and density
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    # Generate a starting solution using the greedy approach
    starting_solution = greedy_solution(i, instance)
    # Perform simulated annealing optimization using single mutation
    solution = simulated_annealing(i, instance, single_mutation, starting_solution, initial_temp, final_temp, alpha)
    # Print or log the results: universe size, number of sets, density, validity, and cost of the solution
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))

In [ ]:
# Run Simulated Annealing with multiple mutations, starting from a greedy solution
for i in range(6):
    # Generate sets and costs for the current universe size and density
    instance = generate_sets_and_costs(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i])
    # Generate a starting solution using the greedy approach
    starting_solution = greedy_solution(i, instance)
    # Perform simulated annealing optimization using multiple mutations
    solution = simulated_annealing(i, instance, multiple_mutations, starting_solution, initial_temp, final_temp, alpha)
    # Print or log the results: universe size, number of sets, density, validity, and cost of the solution
    ic(UNIVERSE_SIZE[i], NUM_SETS[i], DENSITY[i], valid(instance[0], solution), cost(instance[1], solution))